In [ ]:
import requests
import pandas as pd

# Coimbatore coordinates
lat = 11.0168
lon = 76.9558

# Met.no API endpoint
url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"

# Headers with User-Agent
headers = {
    "User-Agent": "YourAppName/1.0 (your@email.com)"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    # Extract relevant data from 'data' and create DataFrame
    # Save or process the DataFrame as needed
else:
    print(f"Error: {response.status_code}")

In [ ]:
pip install requests pandas


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Coimbatore coordinates
lat = 11.0168
lon = 76.9558

# Met.no API endpoint
url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"

# Headers with User-Agent
headers = {
    "User-Agent": "YourAppName/1.0 (your@email.com)"
}

# Make the API request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    timeseries = data['properties']['timeseries']

    # Extract relevant data
    weather_data = []
    for entry in timeseries:
        time = entry['time']
        details = entry['data']['instant']['details']
        weather_data.append({
            "datetime": time,
            "temperature": details.get("air_temperature"),
            "humidity": details.get("relative_humidity"),
            "cloud_cover": details.get("cloud_area_fraction")
        })

    # Convert to DataFrame
    df = pd.DataFrame(weather_data)

    # Save to CSV
    df.to_csv("coimbatore_weather_forecast.csv", index=False)
    print("Weather data saved to coimbatore_weather_forecast.csv")
else:
    print(f"Error: {response.status_code}")


Weather data saved to coimbatore_weather_forecast.csv


In [ ]:
import pandas as pd

# Load the data
df = pd.read_csv("coimbatore_weather_forecast.csv")

# Display the first few rows
print(df.head())


               datetime  temperature  humidity  cloud_cover
0  2025-04-09T06:00:00Z         33.0      43.6        100.0
1  2025-04-09T07:00:00Z         34.4      35.1         97.7
2  2025-04-09T08:00:00Z         35.2      31.3         97.7
3  2025-04-09T09:00:00Z         35.7      27.4         86.7
4  2025-04-09T10:00:00Z         35.8      25.3         76.6


In [ ]:
import pandas as pd

# Load your existing weather forecast CSV
df = pd.read_csv("coimbatore_weather_forecast.csv")
df['datetime'] = pd.to_datetime(df['datetime'])

# Fill missing cloud cover with 0 (clear sky)
df['cloud_cover'] = df['cloud_cover'].fillna(0)

# Step 1: Estimate solar irradiance from cloud cover
# Assume max irradiance = 1000 W/m² (typical max)
df['estimated_irradiance'] = 1000 * (1 - df['cloud_cover'] / 100)

# Step 2: Panel specs
panel_area = 1.94  # m²
efficiency = 0.206
num_panels = 3

# Step 3: Power output (W) from all panels
df['power_W'] = df['estimated_irradiance'] * panel_area * efficiency * num_panels

# Step 4: Convert to energy per hour (kWh)
df['energy_kWh'] = df['power_W'] / 1000

# Step 5: Final output
df_out = df[['datetime', 'cloud_cover', 'estimated_irradiance', 'energy_kWh']]
df_out.to_csv("solar_output_3_panels.csv", index=False)
print("✅ Saved: solar_output_3_panels.csv")
print(df_out.head(10))


✅ Saved: solar_output_3_panels.csv
                   datetime  cloud_cover  estimated_irradiance  energy_kWh
0 2025-04-09 08:00:00+00:00        100.0                   0.0    0.000000
1 2025-04-09 09:00:00+00:00         98.4                  16.0    0.019183
2 2025-04-09 10:00:00+00:00         23.4                 766.0    0.918373
3 2025-04-09 11:00:00+00:00         16.4                 836.0    1.002297
4 2025-04-09 12:00:00+00:00          7.0                 930.0    1.114996
5 2025-04-09 13:00:00+00:00         34.4                 656.0    0.786492
6 2025-04-09 14:00:00+00:00         87.5                 125.0    0.149865
7 2025-04-09 15:00:00+00:00         79.7                 203.0    0.243381
8 2025-04-09 16:00:00+00:00         35.9                 641.0    0.768508
9 2025-04-09 17:00:00+00:00         30.5                 695.0    0.833249


In [ ]:
from google.colab import files
files.download("solar_output_3_panels.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load your solar panel output file
df = pd.read_csv("solar_output_3_panels.csv")

# Convert datetime and extract useful time features
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month

# Optional: Filter daytime hours (e.g., 6 AM to 6 PM)
df = df[(df['hour'] >= 6) & (df['hour'] <= 18)]

# Features for prediction
features = ['hour', 'day', 'month', 'cloud_cover', 'estimated_irradiance']
target = 'energy_kWh'


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Split data
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"✅ Model trained. MAE: {mae:.3f} kWh")


✅ Model trained. MAE: 0.014 kWh


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predict on test set
y_pred = model.predict(X_test)

# 📏 Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 Model Performance:")
print(f"✅ MAE  (Mean Absolute Error):      {mae:.3f} kWh")
print(f"✅ RMSE (Root Mean Squared Error):  {rmse:.3f} kWh")
print(f"✅ R²   (R-squared):                {r2:.3f}")


📊 Model Performance:
✅ MAE  (Mean Absolute Error):      0.014 kWh
✅ RMSE (Root Mean Squared Error):  0.019 kWh
✅ R²   (R-squared):                0.997


In [ ]:
# Sample input: 12 PM, 10% cloud, 950 irradiance
sample_input = pd.DataFrame([{
    'hour': 12,
    'day': 10,
    'month': 4,
    'cloud_cover': 10,
    'estimated_irradiance': 950
}])

predicted_energy = model.predict(sample_input)[0]
print(f"🌞 Predicted Solar Energy (1 hour): {predicted_energy:.2f} kWh")


🌞 Predicted Solar Energy (1 hour): 1.12 kWh


In [ ]:
import joblib
joblib.dump(model, 'solar_energy_predictor.pkl')
print("✅ Model saved as solar_energy_predictor.pkl")


✅ Model saved as solar_energy_predictor.pkl


In [ ]:
from google.colab import files
files.download('solar_energy_predictor.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>